In [1]:
from collections import namedtuple, deque
from tqdm import tqdm

import gymnasium as gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical, Normal

# allow PyTorch throw errors as soon as a NaN gradient is detected
torch.autograd.set_detect_anomaly(True)

# if GPU is to be used
""" ref: M1 GPU support
https://developer.apple.com/metal/pytorch/
https://pytorch.org/blog/introducing-accelerated-pytorch-training-on-mac/
"""
_device_name = "cuda" if torch.cuda.is_available() else "cpu"
_device_name = "mps" if torch.backends.mps.is_available() else _device_name
device = torch.device(_device_name)

# 1. MountainCar-Continuous Environment

In [2]:
""" Environment information
ref: https://github.com/openai/gym/blob/master/gym/envs/classic_control/continuous_mountain_car.py#L27

Observation Space
    The observation is a `ndarray` with shape `(2,)` where the elements correspond to the following:
    | Num | Observation                          | Min  | Max | Unit         |
    |-----|--------------------------------------|------|-----|--------------|
    | 0   | position of the car along the x-axis | -Inf | Inf | position (m) |
    | 1   | velocity of the car                  | -Inf | Inf | position (m) |
Action Space
    The action is a `ndarray` with shape `(1,)`, representing the directional force applied on the car.
    The action is clipped in the range `[-1,1]` and multiplied by a power of 0.0015.
Reward
    A negative reward of *-0.1 * action<sup>2</sup>* is received at each timestep to penalise for
    taking actions of large magnitude. If the mountain car reaches the goal then a positive reward of +100
    is added to the negative reward for that timestep.
Starting State
    The position of the car is assigned a uniform random value in `[-0.6 , -0.4]`.
    The starting velocity of the car is always assigned to 0.
Episode End
    The episode ends if either of the following happens:
    1. Termination: The position of the car is greater than or equal to 0.45 (the goal position on top of the right hill)
    2. Truncation: The length of the episode is 999.
"""
env = gym.make('MountainCarContinuous-v0', render_mode="human")
# env.reset()
# env.render()

In [3]:
from collections import deque
from gym import spaces
import numpy as np

class ConcatObs(gym.Wrapper):
    def __init__(self, env, k):
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=((k,) + shp), dtype=env.observation_space.dtype)

    def reset(self, options=None):
        ob, _ = self.env.reset(options=options)
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, _, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        return np.array(self.frames)

env = gym.make('MountainCarContinuous-v0', render_mode="human")
concat_env = ConcatObs(env, 4)
concat_env.reset()

2023-07-25 23:04:35.393 Python[97060:4524954] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/rd/xnkdmhqx0c19zv0s0z9zcc9c0000gn/T/org.python.python.savedState


array([[-0.5914051,  0.       ],
       [-0.5914051,  0.       ],
       [-0.5914051,  0.       ],
       [-0.5914051,  0.       ]], dtype=float32)

Sample environment image

<img width=300 src="mountain_car.png" />

In [4]:
print("observation_space: ", concat_env.observation_space)

obs = concat_env.reset(options={"low": -1.0, "high": 0.3})  # set random initial position [-1, 0.3]
print("sample obs: ", obs)

print("action_space: ", concat_env.action_space)
print("sample action: ", concat_env.action_space.sample())

observation_space:  Box([[-inf -inf]
 [-inf -inf]
 [-inf -inf]
 [-inf -inf]], [[inf inf]
 [inf inf]
 [inf inf]
 [inf inf]], (4, 2), float32)
sample obs:  [[0.08869325 0.        ]
 [0.08869325 0.        ]
 [0.08869325 0.        ]
 [0.08869325 0.        ]]
action_space:  Box(-1.0, 1.0, (1,), float32)
sample action:  [-0.21842176]


In [5]:
# sample step
new_state, reward, done, _, info = env.step(env.action_space.sample())
print("sample step: ", (new_state, reward, done, info))

sample step:  (array([ 0.08576267, -0.00293058], dtype=float32), -0.011951178394061035, False, {})


# 2. REINFORCE with continuous action space

In [6]:
class VanillaNN(nn.Module):
    def __init__(self, n_observations, n_actions):
        super(VanillaNN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.mu_layer = nn.Linear(128, n_actions)
        self.sigma_layer = nn.Linear(128, n_actions)

    def forward(self, x):
        # flatten 2x2D array to 1D array first
        x = x.flatten()
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        mu = self.mu_layer(x)
        sigma = torch.exp(self.sigma_layer(x))  # Ensure positive standard deviation
        return mu, sigma

In [7]:
sample_nn = VanillaNN(concat_env.observation_space.shape[0]*concat_env.observation_space.shape[1], concat_env.action_space.shape[0])

state = concat_env.reset()
_state = torch.tensor(state, dtype=float).float()

mu, sigma = sample_nn(_state)

print("state: ", state)
print("mu pred: {}, sigma pred: {}".format(mu, sigma))

state:  [[-0.5267669  0.       ]
 [-0.5267669  0.       ]
 [-0.5267669  0.       ]
 [-0.5267669  0.       ]]
mu pred: tensor([-0.1702], grad_fn=<AddBackward0>), sigma pred: tensor([0.9718], grad_fn=<ExpBackward0>)


In [11]:
class ReinforceContinuous:
    def __init__(self, env, gamma=0.99, learning_rate=1e-3):
        self.env = env
        self.gamma = gamma
        self.learning_rate = learning_rate

        # get number of actions and observations
        self._n_observations = self.env.observation_space.shape[0] * self.env.observation_space.shape[1]
        self._n_actions = self.env.action_space.shape[0]

        # setup NN model
        self.policy_net = VanillaNN(self._n_observations, self._n_actions).to(device)

        # setup optimizer
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.learning_rate)
        self.esp = np.finfo(np.float32).eps.item() # prevent division by zero
        
        # set random initial position to improve exploration
        self.initial_options = {"low": -1.0, "high": 0.3}  # default: {"low": -0.6, "high": -0.4}

    def choose_action(self, state):
        # forward pass to get estimate parameters (mu and sigma) of 1 action
        _state = torch.from_numpy(state).float().to(device)
        mu, sigma = self.policy_net(_state)
        
        if torch.isnan(mu).any() or torch.isnan(sigma).any():
            print("found erorr at state =", _state)
            print("NaN detected in mu or sigma")
        
        if mu is None or sigma is None:
            print("found erorr at state =", _state)
            print("mu: {}, sigma: {}".format(mu, sigma))
        
        # calculate normal distribution from mean and std
        m = Normal(mu, torch.exp(sigma) + self.esp)
        
        # sample action and calculate log probability
        selected_action = m.sample()
        log_prop = m.log_prob(selected_action)
        
        return selected_action, log_prop

    def get_trajectory(self, max_steps=500, render=False):
        trajectory = list()
        done = False  # incase of early loop-termination (max_steps) before the environment terminated
        state = self.env.reset(options=self.initial_options)

        for _ in range(max_steps):
            # choose and take action based on normal distribution
            _selected_action, log_prop = self.choose_action(state)
            
            # apply clip to action to ensure it is within the action space
            selected_action = np.clip(_selected_action.item(), -1, 1)
            
            next_state, reward, done, _ = self.env.step([selected_action])
            trajectory.append((state, selected_action, reward, log_prop))
            
            state = next_state
            
            if render:
                self.env.render()

            if done:
                break

        return trajectory, done

    def train(self, num_episodes=1000, max_steps=500, SEED=123):
        self.env.action_space.seed(SEED)
        _ = torch.manual_seed(SEED)
        self.num_returns = 0
        self.sum_returns = 0
        self.sum_returns_squared = 0
        
        # track recent last return to identify if the environment is solved
        running_rewards = deque(maxlen=100)
        
        for episode in tqdm(range(num_episodes)):
            # get sample trajectory from the policy network
            trajectory, _ = self.get_trajectory(max_steps=max_steps)
            
            # prepare data for training
            states, actions, rewards, log_props = zip(*trajectory)
            states = torch.Tensor(np.array(states))
            actions = torch.Tensor(np.array(actions))
            log_props = torch.stack(log_props)
            
            # calculate expected discounted returns
            expected_returns_batch = list()
            for idx in range(len(rewards)):
                discounted_rewards = [self.gamma**i * reward for i, reward in enumerate(rewards[idx:])]
                expected_returns_batch.append(sum(discounted_rewards))
            
            # normalize and reformat expected returns
            expected_returns = torch.FloatTensor(expected_returns_batch)

            self.num_returns += len(expected_returns)
            self.sum_returns += sum(expected_returns)
            self.sum_returns_squared += sum(expected_returns**2)
            
            _return_mean = self.sum_returns / self.num_returns
            _return_std_dev = np.sqrt(self.sum_returns_squared / self.num_returns - _return_mean**2)
            normalized_expected_returns = (expected_returns - _return_mean) / (_return_std_dev + self.esp)
            
            # calculate loss
            loss = - torch.sum(log_props * normalized_expected_returns.to(device))
            
            # log results
            _episode_reward = sum(rewards)
            running_rewards.append(_episode_reward)
            running_mean = np.array(running_rewards).mean()
            running_std_dev = np.array(running_rewards).std()
            running_max = np.array(running_rewards).max()
            if (episode+1) % 25 == 0:
                print(f"Episode {episode+1}\taverage reward: {running_mean:.2f}, std dev: {running_std_dev:.2f}, max: {running_max:.2f}")
                
            if running_mean > 75 and len(running_rewards) >= 100:
                print(f"Solved! Running reward is now {running_mean:.2f}")
                print(f"Episode {episode+1}\taverage reward: {running_mean:.2f}, std dev: {running_std_dev:.2f}, max: {running_max:.2f}")
                break
            
            # optimize the model with backpropagation
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
    def visualize_policy(self, num_episodes=1, max_steps=500):
        for _ in range(num_episodes):
            trajectory, done = self.get_trajectory(max_steps=max_steps, render=True)
            
    def evaluate_policy(self, num_episodes=100, max_steps=500):
        win = 0
        for _ in range(num_episodes):
            trajectory, done = self.get_trajectory(max_steps=max_steps)
            if done and trajectory[-1][2] >= 80:
                win += 1
        return win / num_episodes

In [12]:
env = gym.make("MountainCarContinuous-v0", render_mode="human")
concat_env = ConcatObs(env, 8)
reinforce_agent = ReinforceContinuous(concat_env)
reinforce_agent.train(num_episodes=int(1e3), max_steps=500)

  0%|          | 0/1000 [00:00<?, ?it/s]

  2%|▏         | 24/1000 [08:47<5:54:57, 21.82s/it]

Episode 25	average reward: -36.26, std dev: 26.06, max: 91.35


  5%|▍         | 49/1000 [18:06<6:06:14, 23.11s/it]

Episode 50	average reward: -32.95, std dev: 31.97, max: 91.35


...

Episode 675	average reward: 64.52, std dev: 42.79, max: 93.86


 70%|██████▉   | 699/1000 [3:28:36<47:57,  9.56s/it]  

Episode 700	average reward: 72.75, std dev: 35.11, max: 96.57


 71%|███████▏  | 713/1000 [3:30:22<1:24:40, 17.70s/it]

Solved! Running reward is now 75.33
Episode 714	average reward: 75.33, std dev: 31.65, max: 96.62


In [13]:
# test the trained agent
test_episodes = 10
win_rate = reinforce_agent.evaluate_policy(num_episodes=test_episodes, max_steps=999)
print(f'agent win rate: {win_rate*100 :.2f}% from {test_episodes} test episodes')

agent win rate: 100.00% from 10 test episodes


In [14]:
# visualize the trained agent on a separate window
reinforce_agent.visualize_policy(num_episodes=10, max_steps=999)